In [44]:
%%writefile ~/tmp/test01.py
#!/usr/bin/env python

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); 

swap = list()
final_swap = list()

if rank == 0:
    swap.append(5); swap.append(1); swap.append(2);
    final_swap = comm.alltoall(swap)
    final_swap[2] = comm.bcast(final_swap[2], root = 0)
    comm.send(final_swap[2], dest = 2, tag = 43)
    final_swap[0] = comm.recv(source = 1, tag = 43)
    comm.reduce(final_swap[1], op = MPI.MAX, root = 1)
if rank == 1:  
    swap.append(3); swap.append(4); swap.append(5);
    final_swap = comm.alltoall(swap);
    final_swap[0] = comm.bcast(final_swap[2], root = 0)
    comm.send(final_swap[2], dest = 0, tag = 43)
    final_swap[2] = comm.reduce(final_swap[1], op = MPI.MAX, root = 1)
if rank == 2: 
    swap.append(6); swap.append(7); swap.append(8);
    final_swap = comm.alltoall(swap)
    final_swap[2] = comm.bcast(final_swap[2], root = 0)
    final_swap[0] = comm.recv(source = 0, tag = 43)
    comm.reduce(final_swap[1], op = MPI.MAX, root = 1)

print ("Rank: ", rank, " Swap: ", final_swap)

Overwriting /home/lngo/tmp/test01.py


In [45]:
!chmod 755 ~/tmp/test01.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 3 ~/tmp/test01.py

Rank:  0  Swap:  [7, 3, 6]
Rank:  1  Swap:  [6, 4, 5]
Rank:  2  Swap:  [6, 5, 6]


In [48]:
%%writefile ~/tmp/test02.py
#!/usr/bin/env python
# workdist.py
from mpi4py import MPI
import random
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name()
A = [2,13,4,3,5,1,0,12,10,8,7,9,11,6,15,14]
B = list()

current_index = rank
for i in range(0,4):
    B.append(A[current_index])
    current_index += 4
    
print ("Process %s has elements %s" % (rank, B))

Overwriting /home/lngo/tmp/test02.py


In [49]:
!chmod 755 ~/tmp/test02.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 ~/tmp/test02.py

Process 0 has elements [2, 5, 10, 11]
Process 1 has elements [13, 1, 8, 6]
Process 2 has elements [4, 0, 7, 15]
Process 3 has elements [3, 12, 9, 14]


In [3]:
%%writefile ~/tmp/test03.py
#!/usr/bin/env python
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD; rank = comm.Get_rank()
if rank == 0:
  x = numpy.linspace(0,80,9)
  print (x)
else:
  x = None
if rank == 1:
  xlocal = numpy.zeros(7)
else:
  xlocal = numpy.zeros(1)

comm.Scatterv([x,(1,7,1),(0,1,8),MPI.DOUBLE],xlocal); 
if rank == 0:
  xGathered = numpy.zeros(9)
else:
  xGathered = None

comm.Gatherv(xlocal,[xGathered,(1,7,1),(0,1,1),MPI.DOUBLE])
print ("process " + str(rank) + " has " +str(xGathered))

Overwriting /home/lngo/tmp/test03.py


In [4]:
!chmod 755 ~/tmp/test03.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 3 ~/tmp/test03.py

[  0.  10.  20.  30.  40.  50.  60.  70.  80.]
process 0 has [  0.  80.  20.  30.  40.  50.  60.  70.   0.]
process 1 has None
process 2 has None
